In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from tensorflow.keras.layers import ReLU, concatenate

In [2]:
path='/content/drive/MyDrive/assignment 1/annotations.csv'
relative_path='/content/drive/MyDrive/assignment 1/train/train/'

In [3]:
def load_data(path,relative_path):
    df=pd.read_csv(path)
    data=[]
    for filename in df['fname']:
        address=relative_path+filename
        data.append(np.load(address))
    label=df['label']
    
    return data,label

In [4]:
data,label=load_data(path,relative_path)

In [5]:
mx_len=0
mn_len=100000
tot=0
for spectrograms in data:
    mx_len=max(mx_len,spectrograms.shape[2])
    mn_len=min(mn_len,spectrograms.shape[2])
    tot+=spectrograms.shape[2]
tot,mn_len,mx_len

(593504, 29, 2584)

In [6]:
def list_to_np(data,max_len):
    final_data=[]
    for spectrogram in data:
        x=spectrogram.flatten()
        y=np.zeros((max_len-spectrogram.shape[2])*128)
        y=np.append(y,x)
        final_data.append(y)
    return np.array(final_data)

In [7]:
data=list_to_np(data,mx_len)

In [8]:
def mfcc(data,mean=1):
  p=[]
  for data in data:
  
    mfcc_features = librosa.feature.mfcc(y=data,sr = 44100,  n_fft = 2048, hop_length = 512,  n_mfcc=128 , window ="hann")
    mfcc_mean = np.mean(mfcc_features,axis=1)
    mfcc_scaled = mfcc_mean / np.max(np.abs(mfcc_mean))

    p.append(data)
  return np.array(p)


In [9]:
mfcc_data=mfcc(data)

In [10]:
label=pd.get_dummies(label)
train_x,test_x,train_y,test_y=train_test_split(mfcc_data,label,test_size=0.2,random_state=28)

In [11]:
model1 = Sequential([
    Dense(64, activation='relu', input_shape = train_x[0].shape),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(10, activation='softmax')
])
print(model1.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                21168192  
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 10)                170       
                                                                 
Total params: 21,170,970
Trainable params: 21,170,970
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
model1.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['acc'])

In [13]:
hist = model1.fit(train_x, train_y, epochs=500, batch_size=32)

Epoch 1/1000
25/25 [==============================] - 4s 25ms/step - loss: 10.2524 - acc: 0.1262
Epoch 2/1000
25/25 [==============================] - 1s 25ms/step - loss: 2.2488 - acc: 0.1550
Epoch 3/1000
25/25 [==============================] - 1s 24ms/step - loss: 2.2075 - acc: 0.1587
Epoch 4/1000
25/25 [==============================] - 1s 23ms/step - loss: 2.1817 - acc: 0.1737
Epoch 5/1000
25/25 [==============================] - 1s 24ms/step - loss: 2.1546 - acc: 0.1825
Epoch 6/1000
25/25 [==============================] - 1s 24ms/step - loss: 2.1390 - acc: 0.1863
Epoch 7/1000
25/25 [==============================] - 1s 24ms/step - loss: 2.0742 - acc: 0.2212
Epoch 8/1000
25/25 [==============================] - 1s 24ms/step - loss: 2.0179 - acc: 0.2325
Epoch 9/1000
25/25 [==============================] - 1s 25ms/step - loss: 1.9890 - acc: 0.2512
Epoch 10/1000
25/25 [==============================] - 1s 24ms/step - loss: 1.9608 - acc: 0.2700
Epoch 11/1000
25/25 [=================

KeyboardInterrupt: ignored

In [14]:
print(model1.evaluate(test_x,test_y))

7/7 [==============================] - 0s 18ms/step - loss: 4.1073 - acc: 0.5800
[4.107264518737793, 0.5799999833106995]
